<style>
#notebook-container{
    box-shadow: none !important;
}

.container {
    width: 80% !important;
}

.notebook_app {
    background: #fff !important;
}

body > #header {
    background:  #f57b00;
}

.navbar-default {
    background: none;
    border: none;
}

.navbar-default .navbar-nav > li > a, #kernel_indicator {
    color: rgba(255, 255, 255, 0.25);
    border-bottom: 2px solid #f57b00;
    transition: all 0.25s;
}

.navbar-default .navbar-nav > li > a:hover, #kernel_indicator:hover {
    border-bottom: 2px solid #fff;
    color: rgba(255, 255, 255, 1);
}

div.input_area {
    border: none;
    border-radius: 0;
    background: #f7f7f7;
    line-height: 1.5em;
    margin: 0.5em 0;
    padding: 0;
}

div.cell {
    transition: all 0.25s;
    border: none;
    position: relative;
    top: 0;
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border: none;
    background: transparent;
    box-shadow: 0 6px 18px #aaa;
    z-index: 10;
    top: -10px;
}


div#pager {
    opacity: 0.85;
    z-index: 9999;
}

.navbar-default .navbar-nav > .open > a, .navbar-default .navbar-nav > .open > a:hover, .navbar-default .navbar-nav > .open > a:focus {
    color: #fff;
    background-color: transparent;
    border-bottom: 2px solid #fff;
}

.dropdown-menu {
    z-index: 999999 !important;
    background-color: #f57b00;
    opacity: 0.95;
}

.dropdown-menu > li > a {
    color: #fff;
}

.dropdown-menu > .disabled > a, .dropdown-menu > .disabled > a:hover, .dropdown-menu > .disabled > a:focus {
    color: rgba(255, 255, 255, 0.25);
}

.navbar-nav > li > .dropdown-menu {
    border: none;
    box-shadow: none;
}

div.output_wrapper {
    background: #eee;
}

div.cell.unselected div.output_area{
    box-shadow: inset 0 0 25px #aaa;
    padding: 1em 0;
    overflow-x: auto;
    transition: all 0.25s;
}

div.cell.selected .output_area {
    box-shadow: inset 0 0 5px #aaa;
    padding: 0.5em 0;
    overflow-x: auto;
}

div.cell.selected .div.output_scroll {
    box-shadow: none;
}

div.output_wrapper {
    margin: 0 0 1em;
    transition: all 0.25s;
}

div.cell.selected .output_wrapper {
    margin: 0;
}

.dataframe {
    background: #fff;
    box-shadow: 0px 1px 2px #bbb;
}

.dataframe thead th, .dataframe tbody td {
    text-align: right;
    padding: 1em;
}

.output, div.output_scroll {
    box-shadow: none;
}

.rendered_html pre code {
    background: #f4f4f4;
    border: 1px solid #ddd;
    border-left: 3px solid #2a7bbd;
    color: #444;
    page-break-inside: avoid;
    font-family: monospace;
    font-size: 15px;
    line-height: 1.6;
    margin-bottom: 1.6em;
    max-width: 100%;
    overflow: auto;
    padding: 1em 1.5em;
    display: block;
    word-wrap: break-word;
}

h1, .h1 {
    font-size: 33px;
    font-family: "Trebuchet MS";
    font-size: 2.5em !important;
    color: #2a7bbd;
}
</style>

# Expected max score in a group of $n$ people

This is inspired by [this tweet](https://twitter.com/weijima01/status/1316690215839334403) by Wei Ji Ma, where he makes the following claim:

> Let’s say group A has 10 people, group B has 2. Each of the 12 people randomly gets assigned a number between 1 and 100. Then I use the highest number in Group A as the score for Group A and the highest number in Group B as the score for Group B. On average, Group A will score 91.4 and Group B 67.2.

As Twitter is not really the place for long-form proofs, the statement is left unchecked. Ji Ma certainly knows what he is talking about, but lets prove the statement to be true or false anyway.

To formalize everything a bit, we'll assume we have $n$ people. Each person gets a score $X_i$, where $i \in \{1, 2, \ldots, n \}$, and each score is uniformally distributed with the possible scores $1, 2, \ldots, 100$. Naturally, the probability of getting any one score is $1 / 100$, and we define $f(x) = 1/100$. However, we are interested in the top score in a group of $n$ people. To this end, define
$$
  Y = \max{\{X_1, X_2, \ldots, X_n \}}.
$$
An important function is statistics is the cumulative probability function, given by $F(a) = P(X \leq a)$ for a random variable $X$. In our case, with $Y$ as defined above, we find
$$
  P(Y \leq a) = P(X_1 \leq a \cap X_2 \leq a \cap \cdots \cap X_n \leq a),
$$
that is, if the maximum score is smaller than $a$, this means _every_ score is smaller than $a$.
As all of our random variables $X_i$ are independent and identically distributed, this means
$$
  P(Y \leq a) = \left[ P(X \leq a) \right]^n,
$$
where $X$ is a uniformally distributed random variable in the range $\{1, 2, \ldots, 100\}$ – i.e. $X$ is a stand-in for $X_i$. Naturally, $P(X \leq a)$ is given by the probability that $X=1$ plus the probability that $X=2$ and so on, all the way until $X=$. By working this out we get
$$
  P(X \leq a) = \sum_{k=1}^{a} \frac{1}{100} = \frac{a}{100},
$$
and so
$$
  P(Y \leq a) = \left(\frac{a}{100}\right)^n.
$$
To find the expected value, defined by
$$
  E[Y] = \sum_{k=1}^{100} k P(Y = k),
$$
notice that
$$
  P(Y=k) = P(Y \leq k) - P(Y \leq k-1) = \frac{k^n - (k-1)^n}{100^n},
$$
so the expected value becomes
$$
  E[Y] = \frac{1}{100^n} \sum_{k=1}^{100} (k^{n+1} - k(k-1)^n).
$$
Expressions like the one above are a nightmare by hand, but pretty straightforward in Python:

In [1]:
def expected_max_score(n):
    """Calculate the expected max score in a group of
    n people where each person is assigned a random score
    between 1 and 100"""
    ans = 0
    for k in range(1, 101):
        ans += k ** (n + 1) - k * (k - 1) ** n
    return ans / 100 ** n

We are now finally ready to verify the claim made in the tweet.

In [2]:
ten_people_max_score = expected_max_score(10)
two_people_max_score = expected_max_score(2)

print(
    f"The expected maximum score in a group of ten people is {ten_people_max_score:.1f}."
)
print(
    f"The expected maximum score in a group of two people is {two_people_max_score:.1f}."
)

The expected maximum score in a group of ten people is 91.4.
The expected maximum score in a group of two people is 67.2.


And there you go! We have verified the claim. Of course, we can also perform a few simulations to become even more confident in our work.

In [3]:
from numpy.random import default_rng

In [4]:
rng = default_rng()
ten_people_max = 0
two_people_max = 0
iterations = 10000
for _ in range(iterations):
    ten_people_max += max(rng.integers(1, 100, size=10, endpoint=True))
    two_people_max += max(rng.integers(1, 100, size=2, endpoint=True))
ten_people_max /= iterations
two_people_max /= iterations
print(
    f"Over {iterations} iterations, the group of ten people had an average max score of {ten_people_max:.1f}."
)
print(f"The group of two people had an average max score of {two_people_max:.1f}.")

Over 10000 iterations, the group of ten people had an average max score of 91.4.
The group of two people had an average max score of 67.0.


Pretty good! This is also a good time to remind ourselves of what the expected value actually is – it is the average value of a random variable as the number of trials approach infinity. In other words, the fact that there is a minor inconsitency between our simulation and the analytical value does not mean that we are wrong, it just means i didn't have the time to do an infinite number of iterations this morning.

Do you like this notebook? Do you hate it? [Let me know](twitter.com/satansax).